# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0814 23:37:07.901000 1503690 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:37:07.901000 1503690 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 23:37:09] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=38181, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=20124

[2025-08-14 23:37:09] Using default HuggingFace chat template with detected content format: string


W0814 23:37:17.629000 1504519 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:37:17.629000 1504519 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0814 23:37:17.667000 1504520 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:37:17.667000 1504520 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 23:37:18] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-14 23:37:18] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=32.47 GB): 100%|██████████| 4/4 [00:09<00:00,  2.47s/it]
[2025-08-14 23:37:36] Capture draft cuda graph end. Time elapsed: 10.30 s. mem usage=-20.93 GB. avail mem=53.64 GB.
[2025-08-14 23:37:36] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=53.64 GB


Capturing batches (bs=1 avail_mem=53.56 GB): 100%|██████████| 4/4 [00:00<00:00, 112.17it/s]
[2025-08-14 23:37:37] Capture draft extend cuda graph end. Time elapsed: 0.68 s. mem usage=0.08 GB. avail mem=53.56 GB.
[2025-08-14 23:37:37] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=53.56 GB
[2025-08-14 23:37:37] INFO:     Started server process [1503690]
[2025-08-14 23:37:37] INFO:     Waiting for application startup.


[2025-08-14 23:37:37] INFO:     Application startup complete.
[2025-08-14 23:37:37] INFO:     Uvicorn running on http://127.0.0.1:38181 (Press CTRL+C to quit)


[2025-08-14 23:37:37] INFO:     127.0.0.1:48116 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-14 23:37:38] INFO:     127.0.0.1:48122 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-14 23:37:38] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-14 23:37:38] INFO:     127.0.0.1:48128 - "POST /generate HTTP/1.1" 200 OK
[2025-08-14 23:37:38] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-14 23:37:43] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-14 23:37:43] INFO:     127.0.0.1:45764 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0814 23:37:49.657000 1507813 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:37:49.657000 1507813 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 23:37:50] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=38075, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=3750587

[2025-08-14 23:37:51] Using default HuggingFace chat template with detected content format: string


W0814 23:37:58.047000 1508353 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:37:58.047000 1508353 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0814 23:37:58.097000 1508354 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:37:58.097000 1508354 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 23:37:59] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-14 23:37:59] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=36.11 GB): 100%|██████████| 4/4 [00:00<00:00, 45.09it/s]
[2025-08-14 23:39:05] Capture draft extend cuda graph end. Time elapsed: 1.03 s. mem usage=0.10 GB. avail mem=36.11 GB.
[2025-08-14 23:39:05] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=36.11 GB


[2025-08-14 23:39:05] INFO:     Started server process [1507813]
[2025-08-14 23:39:05] INFO:     Waiting for application startup.
[2025-08-14 23:39:05] INFO:     Application startup complete.
[2025-08-14 23:39:05] INFO:     Uvicorn running on http://127.0.0.1:38075 (Press CTRL+C to quit)


[2025-08-14 23:39:05] INFO:     127.0.0.1:43964 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-14 23:39:06] INFO:     127.0.0.1:43972 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-14 23:39:06] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-14 23:39:08] INFO:     127.0.0.1:43982 - "POST /generate HTTP/1.1" 200 OK
[2025-08-14 23:39:08] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-14 23:39:10] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-14 23:39:10] INFO:     127.0.0.1:43984 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0814 23:39:20.144000 1517104 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:39:20.144000 1517104 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 23:39:21] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=36013, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-08-14 23:39:21] Using default HuggingFace chat template with detected content format: string


W0814 23:39:28.213000 1517929 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:39:28.213000 1517929 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0814 23:39:28.798000 1517928 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:39:28.798000 1517928 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

Capturing batches (bs=1 avail_mem=6.89 GB): 100%|██████████| 4/4 [00:06<00:00,  1.61s/it]
[2025-08-14 23:39:57] Capture draft cuda graph end. Time elapsed: 6.90 s. mem usage=42.90 GB. avail mem=6.13 GB.
[2025-08-14 23:39:57] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=6.13 GB


Capturing batches (bs=1 avail_mem=5.83 GB): 100%|██████████| 4/4 [00:00<00:00, 35.94it/s]
[2025-08-14 23:39:58] Capture draft extend cuda graph end. Time elapsed: 0.83 s. mem usage=0.30 GB. avail mem=5.83 GB.
[2025-08-14 23:39:58] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=5.83 GB


[2025-08-14 23:39:59] INFO:     Started server process [1517104]
[2025-08-14 23:39:59] INFO:     Waiting for application startup.
[2025-08-14 23:39:59] INFO:     Application startup complete.
[2025-08-14 23:39:59] INFO:     Uvicorn running on http://127.0.0.1:36013 (Press CTRL+C to quit)


[2025-08-14 23:40:00] INFO:     127.0.0.1:40650 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-14 23:40:00] INFO:     127.0.0.1:40666 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-14 23:40:00] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-14 23:40:00] INFO:     127.0.0.1:40668 - "POST /generate HTTP/1.1" 200 OK
[2025-08-14 23:40:00] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-14 23:40:05] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-14 23:40:05] INFO:     127.0.0.1:33920 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0814 23:40:12.474000 1521166 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:40:12.474000 1521166 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 23:40:13] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=33097, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_see

[2025-08-14 23:40:14] Using default HuggingFace chat template with detected content format: string


W0814 23:40:20.672000 1521408 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:40:20.672000 1521408 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0814 23:40:20.787000 1521407 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:40:20.787000 1521407 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 23:40:21] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=56.09 GB): 100%|██████████| 4/4 [00:05<00:00,  1.31s/it]
[2025-08-14 23:40:48] Capture draft cuda graph end. Time elapsed: 5.89 s. mem usage=1.49 GB. avail mem=56.05 GB.
[2025-08-14 23:40:48] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=56.05 GB


Capturing batches (bs=1 avail_mem=55.85 GB): 100%|██████████| 4/4 [00:00<00:00, 73.58it/s]
[2025-08-14 23:40:49] Capture draft extend cuda graph end. Time elapsed: 1.07 s. mem usage=0.20 GB. avail mem=55.85 GB.
[2025-08-14 23:40:49] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=55.85 GB


[2025-08-14 23:40:50] INFO:     Started server process [1521166]
[2025-08-14 23:40:50] INFO:     Waiting for application startup.
[2025-08-14 23:40:50] INFO:     Application startup complete.
[2025-08-14 23:40:50] INFO:     Uvicorn running on http://127.0.0.1:33097 (Press CTRL+C to quit)


[2025-08-14 23:40:51] INFO:     127.0.0.1:50754 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-14 23:40:51] INFO:     127.0.0.1:50768 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-14 23:40:51] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-14 23:40:51] INFO:     127.0.0.1:50770 - "POST /generate HTTP/1.1" 200 OK
[2025-08-14 23:40:51] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-14 23:40:56] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-14 23:40:56] INFO:     127.0.0.1:53554 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0814 23:41:02.905000 1524221 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:41:02.905000 1524221 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 23:41:04] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=30105, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=734214016, constrained_jso

[2025-08-14 23:41:05] Using default HuggingFace chat template with detected content format: string


W0814 23:41:13.002000 1524766 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:41:13.002000 1524766 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0814 23:41:13.011000 1524767 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0814 23:41:13.011000 1524767 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-14 23:41:14] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-14 23:41:14] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=59.45 GB):  50%|█████     | 2/4 [00:01<00:01,  1.48it/s]

Capturing batches (bs=1 avail_mem=59.45 GB): 100%|██████████| 4/4 [00:02<00:00,  1.93it/s]
[2025-08-14 23:41:26] Capture draft cuda graph end. Time elapsed: 2.56 s. mem usage=-4.99 GB. avail mem=59.45 GB.
[2025-08-14 23:41:26] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=59.45 GB


Capturing batches (bs=1 avail_mem=59.35 GB): 100%|██████████| 4/4 [00:00<00:00, 23.59it/s]
[2025-08-14 23:41:27] Capture draft extend cuda graph end. Time elapsed: 1.24 s. mem usage=0.09 GB. avail mem=59.35 GB.
[2025-08-14 23:41:27] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=59.35 GB


[2025-08-14 23:41:28] INFO:     Started server process [1524221]
[2025-08-14 23:41:28] INFO:     Waiting for application startup.
[2025-08-14 23:41:28] INFO:     Application startup complete.
[2025-08-14 23:41:28] INFO:     Uvicorn running on http://0.0.0.0:30105 (Press CTRL+C to quit)
[2025-08-14 23:41:28] INFO:     127.0.0.1:34806 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-14 23:41:29] INFO:     127.0.0.1:34810 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-14 23:41:29] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-14 23:41:29] INFO:     127.0.0.1:34826 - "POST /generate HTTP/1.1" 200 OK
[2025-08-14 23:41:29] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-14 23:41:33] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-14 23:41:33] Decode batch. #running-req: 1, #token: 89, token usage: 0.00, accept len: 1.71, cuda graph: True, gen throughput (token/s): 11.22, #queue-req: 0, 


[2025-08-14 23:41:34] Decode batch. #running-req: 1, #token: 155, token usage: 0.01, accept len: 1.65, cuda graph: True, gen throughput (token/s): 143.44, #queue-req: 0, 


[2025-08-14 23:41:34] Decode batch. #running-req: 1, #token: 225, token usage: 0.01, accept len: 1.75, cuda graph: True, gen throughput (token/s): 140.37, #queue-req: 0, 


[2025-08-14 23:41:35] Decode batch. #running-req: 1, #token: 299, token usage: 0.01, accept len: 1.85, cuda graph: True, gen throughput (token/s): 158.24, #queue-req: 0, 


[2025-08-14 23:41:35] Decode batch. #running-req: 1, #token: 365, token usage: 0.02, accept len: 1.65, cuda graph: True, gen throughput (token/s): 134.53, #queue-req: 0, 


[2025-08-14 23:41:36] Decode batch. #running-req: 1, #token: 433, token usage: 0.02, accept len: 1.70, cuda graph: True, gen throughput (token/s): 148.38, #queue-req: 0, 


[2025-08-14 23:41:36] Decode batch. #running-req: 1, #token: 498, token usage: 0.02, accept len: 1.62, cuda graph: True, gen throughput (token/s): 144.59, #queue-req: 0, 


[2025-08-14 23:41:37] INFO:     127.0.0.1:47374 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).